In [83]:
import sys
import pathlib


import requests
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

from utils.download_data import download_parquet

from numpy import source
import pandas as pd
import pickle
import tempfile
from google.cloud import storage
from typing import List
import requests
import pyarrow.parquet as pq
import pyarrow as pa
import pathlib

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error



In [84]:
def get_data(date, taxi_type):
    run_date: str = date or pd.Timestamp.now().date().strftime("%Y-%m-%d")
    date_range = [d.strftime("%Y-%m") for d in pd.date_range(end=run_date, periods=3, freq="MS")]
    
    train_date = date_range[0]
    val_date = date_range[1]
    
    for dt in [train_date, val_date]:
        download_parquet(year_month=dt, taxi_type=taxi_type, directory="data")
        
    return run_date, train_date, val_date

In [107]:
run_date, train_date, val_date = get_data(date="2021-08-10", taxi_type="green")

In [120]:
"_".join(run_date.split("-")[0:2])

'2021_08'

In [121]:
train = pd.read_parquet(f'data/green_tripdata_{"_".join(train_date.split("-")[0:2])}.parquet')
val = pd.read_parquet(f'data/green_tripdata_{"_".join(val_date.split("-")[0:2])}.parquet')


In [126]:
def prep_data(df):
    
    df.columns = df.columns.str.upper()
    df['DURATION'] = (df["LPEP_DROPOFF_DATETIME"] - df["LPEP_PICKUP_DATETIME"])
    df['DURATION'] = df["DURATION"].dt.total_seconds() / 60
    df["ROUTE"] = (df["PULOCATIONID"].astype(str) + "_" + df["DOLOCATIONID"].astype(str)).astype("category")
    
    return df

In [127]:
train = prep_data(train)
val = prep_data(val)

In [128]:
train.head()

,VENDORID,LPEP_PICKUP_DATETIME,LPEP_DROPOFF_DATETIME,STORE_AND_FWD_FLAG,RATECODEID,PULOCATIONID,DOLOCATIONID,PASSENGER_COUNT,TRIP_DISTANCE,FARE_AMOUNT,...,TIP_AMOUNT,TOLLS_AMOUNT,EHAIL_FEE,IMPROVEMENT_SURCHARGE,TOTAL_AMOUNT,PAYMENT_TYPE,TRIP_TYPE,CONGESTION_SURCHARGE,DURATION,ROUTE
0,2,2021-06-01 00:31:19,2021-06-01 00:42:14,N,5.0,168,235,2.0,4.89,15.0,...,4.59,0.0,None,0.3,19.89,1.0,2.0,0.0,10.916667,168_235
1,2,2021-06-01 00:16:07,2021-06-01 00:24:13,N,1.0,82,7,1.0,1.44,7.5,...,0.00,0.0,None,0.3,8.80,2.0,1.0,0.0,8.100000,82_7
2,1,2021-06-01 00:03:58,2021-06-01 00:21:20,N,1.0,244,74,1.0,2.30,13.5,...,0.00,0.0,None,0.3,14.80,2.0,1.0,0.0,17.366667,244_74
3,2,2021-06-01 00:00:57,2021-06-01 00:03:19,N,1.0,254,254,1.0,0.86,4.5,...,0.00,0.0,None,0.3,5.80,2.0,1.0,0.0,2.366667,254_254
4,2,2021-06-01 00:33:22,2021-06-01 00:56:17,N,1.0,127,174,4.0,3.82,18.5,...,0.00,0.0,None,0.3,19.80,1.0,1.0,0.0,22.916667,127_174


In [129]:
categorical = ["ROUTE"]
numerical = ["TRIP_DISTANCE"]
features = categorical + numerical

train_dicts = train[features].to_dict(orient='records')
val_dicts = val[features].to_dict(orient='records')

dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts) 
X_val = dv.transform(val_dicts)

y_train = train["DURATION"].values
y_val = val["DURATION"].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mse = mean_squared_error(y_val, y_pred, squared=False)

mse

77.30413846178791

In [50]:
with open("models/linreg_model.bin", "wb") as f_out:
            pickle.dump((dv, lr), f_out)
    



,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,duration
0,HV0003,B03404,B03404,2022-01-01 00:05:31,2022-01-01 00:05:40,2022-01-01 00:07:24,2022-01-01 00:18:28,170,161,1.18,...,2.75,0.0,0.0,23.03,N,N,,N,N,11.066667
1,HV0003,B03404,B03404,2022-01-01 00:19:27,2022-01-01 00:22:08,2022-01-01 00:22:32,2022-01-01 00:30:12,237,161,0.82,...,2.75,0.0,0.0,12.32,N,N,,N,N,7.666667
2,HV0003,B03404,B03404,2022-01-01 00:43:53,2022-01-01 00:57:37,2022-01-01 00:57:37,2022-01-01 01:07:32,237,161,1.18,...,2.75,0.0,0.0,23.30,N,N,,N,N,9.916667
3,HV0003,B03404,B03404,2022-01-01 00:15:36,2022-01-01 00:17:08,2022-01-01 00:18:02,2022-01-01 00:23:05,262,229,1.65,...,2.75,0.0,0.0,6.30,N,N,,N,N,5.050000
4,HV0003,B03404,B03404,2022-01-01 00:25:45,2022-01-01 00:26:01,2022-01-01 00:28:01,2022-01-01 00:35:42,229,141,1.65,...,2.75,0.0,0.0,7.44,N,N,,N,N,7.683333


In [24]:
categorical = ["DOLocationID","PULocationID"]
df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

In [21]:
taxi_type = "fhvhv"
year_month = "2019-03"
df = pd.read_parquet(f"https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year_month}.parquet", engine="pyarrow")

In [19]:
filename = f"{taxi_type}_{year_month.replace('-','_')}.parquet"

In [20]:
pathlib.Path(f"data/{filename}").is_file()

True

In [29]:
url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year_month}.parquet"
filename = url.split("/")[-1].replace("-","_")

path = pathlib.Path("data")
path.joinpath(filename).is_file()

False

In [18]:
df.to_parquet(f"data/{filename}")

In [37]:
def download_parquet(year_month: str, taxi_type="green", directory="data"):
    
    path = pathlib.Path(directory)
    path.mkdir(parents=True, exist_ok=True)
    
    url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year_month}.parquet"
    filename = url.split("/")[-1].replace("-","_")
    
    if path.joinpath(filename).is_file():
            print(f"File already exists: {filename} \n...Skipping download.")
    else:
        try:
            result = requests.get(url)
        except requests.exceptions.HTTPError as e:
                print("ERROR ".center(90, "-"))
                print(e, file=sys.stderr)
                print("~~> file may not be available on server.")
        except requests.exceptions.RequestException as e:
                print(e, file=sys.stderr)
        
        # Save data to parquet file in local directory and return dataframe
        reader = pa.BufferReader(result.content)
        table = pq.read_table(reader)
        pq.write_table(table, f"{directory}/{filename}")
 

In [38]:
download_parquet(year_month="2019-02", taxi_type="fhvhv", directory="data")

File already exists: fhvhv_tripdata_2019_02.parquet 
...Skipping download.


In [62]:
with tempfile.TemporaryDirectory() as tmpdir:
        storage_client = storage.Client()
        bucket = storage_client.bucket("prefect-artifacts00")
        blob = bucket.blob("fhv/test_blob")
        temp_path = pathlib.Path(tmpdir)
        with open(temp_path.joinpath("dv-test.pkl"), "wb") as f_out:
            pickle.dump(dv, f_out)
        
        for file in temp_path.iterdir():
            print(file.name)

dv-test.pkl


In [30]:
from google.cloud import storage
from typing import  List
def upload_artifacts_gcs(bucket_name: str, source_files: List, destination_blob_name: str):
    """Uploads file to gcs bucket

    Args:
        bucket_name (str): Name of gcs bucket
        source_files (List): List of path(s) of files to upload from local system
        destination_blob_name (str): Name of gcs storage object
    """
    with tempfile.TemporaryDirectory() as tmpdir:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        
        temp_path = pathlib.Path(tmpdir)
        with open(temp_path.joinpath("dv-{run_date}.pkl"), "wb") as f_out:
            pickle.dump(dv, f_out)
        
        for file in temp_path.iterdir():
            blob = bucket.blob("{taxi_type}/{file.name}")
            blob.upload_from_filename(file)


In [ ]:
upload_artifacts_gcs("prefect-artifacts00", )